###### import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import operator
import math

from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from operator import itemgetter

path_train = Path("data/train.pkl")
path_test = Path("data/test.pkl")

train_set = open(path_train,'rb')
test_set = open(path_test, 'rb')
train_dict = pickle.load(train_set)
test_dict = pickle.load(test_set)


train_y=np.array(list(map(itemgetter(1), train_dict)))
#print(train_y[2])
train_x=np.array(list(map(itemgetter(0), train_dict)))

test_y=np.array(list(map(itemgetter(1), test_dict)))
test_x=np.array(list(map(itemgetter(0), test_dict)))
#print(len(test_dict))
#print(len(train_dict))

train_split_x =np.array_split(train_x, 10)
train_split_y =np.array_split(train_y, 10)
#print(train_split_x)

BIAS_T=[]
VAR_T=[]
bias_value=[]
ERROR_T=[]

for i in range(1,21):
	bias=[]
	var=[]
	arr=[]
	
	for j in range(10):
		#numpy array created into 2D array with one column
		y_test=np.reshape(test_y,(-1,1))
		x_test=np.reshape(test_x,(-1,1))
		x_train=np.reshape([train_split_x[j]],(-1,1))
		#print(x_train)
		y_train=np.reshape(train_split_y[j], (-1,1))
		
		polynomial = PolynomialFeatures(degree = i)
		x_train = polynomial.fit_transform(x_train)
		x_test=polynomial.fit_transform(x_test)

		linear = LinearRegression().fit(x_train, y_train)
		predicted_y = linear.predict(x_test)
		arr.append(predicted_y)
		
		#print(arr)
	
	prediction=[]	
	for k in range(10):
		#numpy array being created into 1D array
		prediction.append(arr[k].T[0])
		#print(arr[k].T[0])

	arr=np.array(prediction)
	#print(arr)
	
	mean_bias=0
	mean_var=0
	mse=0
	for k in range(80):
		mean_bias= mean_bias+ ((np.mean(arr.T[k])-test_y[k])**2)
		mean_var= mean_var+ np.mean(np.var(prediction, axis=0))
		mse=mse+ (np.mean((arr.T[k])-test_y[k])**2)

	mean_bias=mean_bias/80
	mean_var=mean_var/80
	mse=mse/80

	bias_only=math.sqrt(mean_bias)

	BIAS_T.append(mean_bias)
	VAR_T.append(mean_var)
	bias_value.append(bias_only)
	#print(BIAS_T)
	ERROR_T.append(mse-mean_bias-mean_var)

Total_error=np.mean(ERROR_T)
#print(Total_error)

print("Degree", "        Bias        ","        Variance         ", "         Error        ")
for i in range(20):
    print(i+1,"      ",bias_value[i],"     ",VAR_T[i], "      ",ERROR_T[i])
    polynomial = PolynomialFeatures(degree = i, include_bias = False)
    plt.plot(VAR_T, color="red")
    plt.plot(BIAS_T, color="blue")
    plt.title('Bias^2  vs  Variance')
    plt.xlabel("Model Complexity")
    plt.ylabel("Error")
    plt.legend(('Bias Squared', 'Variance'), loc='best')
plt.show()

#plotting error
""""
for i in range(20):
   
    polynomial = PolynomialFeatures(degree = i, include_bias = False)
    plt.plot(ERROR_T, color="red")
    plt.xlabel("degree")
    plt.title("Irreduceble error")
   	
plt.show()
"""